In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello World!'

if __name__ == '__main__':
    app.run()

In [1]:
from flask import Flask #간단히 플라스크 서버를 만든다

import urllib.request

app = Flask(__name__)

@app.route("/tospring")
def spring():
    
    return "동현씨 아자아자 파이팅~!~!!~~!"
    
    
if __name__ == '__main__':
    app.run(debug=False,host="127.0.0.1",port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2022 17:30:12] "GET /tospring HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2022 17:30:15] "GET /tospring HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2022 17:34:35] "GET /tospring HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2022 17:39:02] "GET /tospring HTTP/1.1" 200 -


## 중계서버 코드

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/upload', methods = ['GET', 'POST'])
def file_upload():
    if request.method == 'POST': # POST 방식 처리
        f = request.files['file1'] # 받아온 파일 객체 생성
 
        s_filename = secure_filename(f.filename) # 파일명 저장
        file_dir = 'uploads/' + s_filename # 파일을 저장하기 위한 경로 지정
 
        f.save('static/uploads/' + s_filename) # 파일 저장
        files = os.listdir("static/uploads")
 
        upload = {'image': open('static/uploads/' + s_filename, 'rb')} # 업로드하기위한 파일
        res = requests.post('http://127.0.0.1:81/receive', files=upload).json() # JSON 포맷, POST 형식으로 해당 URL에 파일 전송
        machineResult = res['cal_result'] # 받아온 JSON 형식의 response를 처리함
 
        # 데이터베이스 연결을 위한 connect 생성
        conn = mysql.connect()
        cursor = conn.cursor()
        # 파일명과 파일경로를 데이터베이스에 저장함
        sql = "INSERT INTO images (image_name, image_dir) VALUES ('%s', '%s')" % (s_filename, file_dir)
        cursor.execute(sql) # sql문 입력
        data = cursor.fetchall() # 처리 결과
 
        if not data:
            conn.commit() # 수정 내용을 저장함
            return jsonify({"cal_result": machineResult}) # 결과 값 반환
 
        else:
            conn.rollback() # 수정 내용을 저장하지 않음
            return 'upload failed'
 
        # 데이터베이스와 연결 종료
        cursor.close()
        conn.close()

## 머신러닝 서버 코드

In [ ]:
@app.route('/receive', methods=['GET', 'POST'])
def receive():
    if request.method == 'POST':
        f = request.files['image'] # 보낸 파일을 받아옴
        f.save('static/images/' + secure_filename(f.filename)) # 해당 파일 저장
        files = os.listdir("static/images")
 
        remove_background.remove('static/images/' + secure_filename(f.filename)) # 배경 제거
 
        # 머신러닝 결과를 변수에 저장
        predition = model.predict_food_transfer(model_transfer, test_transform, class_names, 'static/images/result.jpg')
 
        os.remove('static/images/' + secure_filename(f.filename))
 
    return jsonify({"cal_result": predition}) # 머신러닝 결과 반환

### 클라이언트 - 서버 - 서버(ai 계산)의 구조로 서비스를 만들려고 한다.

### 그래서 가장 먼저 클라이언트에서 서버로 이미지를 전송하는 기능을 구현했다.

In [1]:
from flask import Flask, render_template, request, redirect, url_for
from flaskext.mysql import MySQL
from werkzeug.utils import secure_filename
 
import os
 
mysql = MySQL()
app = Flask(__name__)
 
# 데이터베이스 값 설정
app.config['MYSQL_DATABASE_USER'] = 'root'
app.config['MYSQL_DATABASE_PASSWORD'] = '0000'
app.config['MYSQL_DATABASE_DB'] = 'image'
app.config['MYSQL_DATABASE_HOST'] = 'localhost'
app.secret_key = "ABCDEFG"
mysql.init_app(app)
 
@app.route('/', methods=['GET', 'POST'])
def main():
    return render_template('main.html')
 
@app.route('/fileUpload', methods = ['GET', 'POST'])
def file_upload():
    if request.method == 'POST':
        f = request.files['file']
        f.save('static/uploads/' + secure_filename(f.filename))
        files = os.listdir("static/uploads")
 
        conn = mysql.connect()
        cursor = conn.cursor()
        # 파일명과 파일경로를 데이터베이스에 저장함
        sql = "INSERT INTO images (image_name, image_dir) VALUES ('%s', '%s')" % (secure_filename(f.filename), 'uploads/'+secure_filename(f.filename))
        cursor.execute(sql)
        data = cursor.fetchall()
 
        if not data:
            conn.commit()
            return redirect(url_for("main"))
 
        else:
            conn.rollback()
            return 'upload failed'
 
        cursor.close()
        conn.close()
 
@app.route('/view', methods = ['GET', 'POST'])
def view():
    conn = mysql.connect()  # DB와 연결
    cursor = conn.cursor()  # connection으로부터 cursor 생성 (데이터베이스의 Fetch 관리)
    sql = "SELECT image_name, image_dir FROM images"  # 실행할 SQL문
    cursor.execute(sql)  # 메소드로 전달해 명령문을 실행
    data = cursor.fetchall()  # 실행한 결과 데이터를 꺼냄
 
    data_list = []
 
    for obj in data:  # 튜플 안의 데이터를 하나씩 조회해서
        data_dic = {  # 딕셔너리 형태로
            # 요소들을 하나씩 넣음
            'name': obj[0],
            'dir': obj[1]
        }
        data_list.append(data_dic)  # 완성된 딕셔너리를 list에 넣음
 
    cursor.close()
    conn.close()
 
    return render_template('view.html', data_list=data_list)  # html을 렌더하며 DB에서 받아온 값들을 넘김
 
if __name__ == '__main__':
    app.run(port=80)

ModuleNotFoundError: No module named 'flaskext'